In [1]:
# Load modules necessary
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
from scipy.signal import detrend

%matplotlib qt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Load sample dataset

##### For EEG dataset, we will use CHB-MIT Scalp EEG Dataset from here: https://physionet.org/content/chbmit/1.0.0/chb01/#files-panel

##### The original dataset contains 22 subjects. However, here in this tutorial we will explore data from Subject 01
##### You can download the dataset from Brightspace

# Matrix multiplications as filters

In [2]:
# Path to the EEG file
eegPath = '../../Datasets/EEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr'

# Load the EEG file using MNE
# MNE has different read formats for different EEG file types
# Here we are using read_raw_edf to read the EEG file
# preload=True loads the data into memory (default is False, which loads the data when needed)
raw = mne.io.read_raw_brainvision(eegPath, preload=True)
elecPos = pd.read_csv('../../Datasets/EEG/sub-01/eeg/sub-01_electrodes.tsv', sep='\t')
# Add fiducials
fiducials = pd.DataFrame({
    'name': ['Nz', 'LPA', 'RPA'],
    'x': [-4.129838157917329e-18, -0.0729282673627754, 0.08278152042487033],
    'y': [0.10011015398430487, 3.008505424862354e-18, -3.414981080487009e-18],
    'z': [-5.7777898331617076e-33, 3.851859888774472e-34, 3.4666738998970245e-33]
})

# Concatenate the original electrode positions with the fiducials
elecPos = pd.concat([elecPos, fiducials], ignore_index=True)

montage = mne.channels.make_dig_montage(
    ch_pos=dict(zip(elecPos['name'], elecPos[['x', 'y', 'z']].values)),
    coord_frame='head'
)
raw.set_montage(montage)

data = raw.get_data()
n_channels = data.shape[0]
n_samples = data.shape[1]

Extracting parameters from ../../Datasets/EEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr...
Setting channel info structure...
Reading 0 ... 244237  =      0.000 ...   976.948 secs...


In [ ]:
# Select an electrode via vector multiplication
selectElecIdx = 2
selectElecVec = np.zeros((n_channels,1))


print(selectElecVec.shape)
print(data.shape)
selectElecVec[selectElecIdx] = 1
selectElecData = (selectElecVec.T @ data)

# print(selectElecVec)
print(selectElecData.shape)
print(selectElecData)

In [ ]:
# Select a subset of electrodes via matrix multiplication
# Let's select channels 1, 3, and 5
selected_channels = [0, 2, 4]
ChannelMultiplier = np.zeros((len(selected_channels), n_channels))
for i, chan in enumerate(selected_channels):
    ChannelMultiplier[i, chan] = 1
# print(ChannelMultiplier)
print(ChannelMultiplier.shape)

print(data.shape)
data_selected = (ChannelMultiplier @ data)

plt.figure()
plt.imshow(ChannelMultiplier, aspect='auto')
plt.show()

In [ ]:
# Average an electrode-ROI via vector multiplication
# Let's average across occipital electrodes
elecList = ['O1', 'Oz', 'O2', 'PO3', 'POz', 'PO4']
# Get indices for the specified electrodes
elecIdx = [elecPos[elecPos['name'] == elec].index[0] for elec in elecList]
elecVec = np.zeros((len(elecIdx), n_channels))
for i, elec in enumerate(elecIdx):
    elecVec[i, elec] = 1/len(elecIdx)
elecData = (elecVec @ data)

# print(elecVec)
print(elecVec.shape)
print(data.shape)

plt.figure()
plt.imshow(elecVec, aspect='auto')
plt.show()


In [5]:
# Channel interpolation via matrix multiplication
# Let's interpolate channel 31 using channels 30 and 32
interp_channel = 30
InterpMultiplier = np.eye(n_channels)
InterpMultiplier[interp_channel, interp_channel-1] = 1/2
InterpMultiplier[interp_channel, interp_channel+1] = 1/2
InterpMultiplier[interp_channel, interp_channel] = 0
print(InterpMultiplier.shape)
print(data.shape)
data_interp = (InterpMultiplier @ data)

plt.figure()
plt.imshow(InterpMultiplier, aspect='auto')
plt.show()

# Plot the interpolated EEG data

f, axs = plt.subplots(3, 1, figsize=(15, 10))
axs = axs.flatten()
for i, chan in enumerate(np.array([29, 30, 31])):
    ax = axs.flatten()[i]
    ax.plot(data[chan, :2000])
    ax.set_title(f"Channel {chan+1}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude")
plt.show()

f, axs = plt.subplots(3, 1, figsize=(15, 10))
axs = axs.flatten()
for i, chan in enumerate(np.array([29, 30, 31])):
    ax = axs.flatten()[i]
    ax.plot(data_interp[chan, :2000])
    ax.set_title(f"Channel {chan+1}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude")
plt.show()


(66, 66)
(66, 244238)


In [6]:
# Fixing swapped electrodes
# Channel 1 and 3 have been plugged into wrong position in the cap
# Swap the channels using matrix multiplication
SwapMultiplier = np.eye(n_channels)
SwapMultiplier[0, 2] = 1
SwapMultiplier[2, 0] = 1
SwapMultiplier[0, 0] = 0
SwapMultiplier[2, 2] = 0

data_swapped = (SwapMultiplier @ data)

print(SwapMultiplier)

plt.figure()
plt.imshow(SwapMultiplier, aspect='auto')
plt.show()

[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [10]:
# Re-referencing data to a single channel using matrix multiplication
# Let's use channel 10 as the reference channel

# Fix this and elaborate on concept of distributitivity and selector matrix by matrix multiplication as well

averages_mat = np.ones(n_channels)/n_channels    ;

reref_data = np.eye(n_channels)@data - averages_mat @ data
# A * X - B* X
# (A - B) * X
reref_data = (np.eye(n_channels)- averages_mat)@data  


print(reref_data)

# ref_channel = 9
# ReferenceMultiplier = np.eye(n_channels)
# ReferenceMultiplier[:, ref_channel] = -1
# ReferenceMultiplier[ref_channel, :] = 0
# # ReferenceMultiplier[ref_channel, ref_channel] = 1
# print(ReferenceMultiplier)
# data_single_ref = (ReferenceMultiplier @ data)

# plt.figure()
# plt.imshow(ReferenceMultiplier, aspect='auto')
# plt.show()
# # # Plot the single re-referenced EEG data
# # f, axs = plt.subplots(nrows, ncols, figsize=(15, 10))
# # axs = axs.flatten()
# # for i in range(n_channels):
#     ax = axs.flatten()[i]
#     ax.plot(data_single_ref[i, :])
#     ax.set_title(f"Channel {i+1}")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("Amplitude")

[[-2.85133927e-06 -1.51774682e-05 -1.29183195e-04 ...  6.22013642e-04
   5.77549271e-04  3.02237040e-04]
 [ 7.11370023e-06 -2.95771210e-05 -1.37599778e-04 ...  6.01960772e-04
   5.05273085e-04  2.69266423e-04]
 [-1.63045931e-06 -1.65571677e-05 -1.27210092e-04 ...  4.52623180e-04
   6.49537071e-04  1.11018035e-04]
 ...
 [ 1.31319676e-02 -1.85885249e-02 -1.32584313e-02 ... -4.22316167e-02
  -3.15536107e-02 -1.87773344e-02]
 [-1.30916710e-02  1.97375423e-02  2.14800652e-02 ...  4.06451036e-04
   2.40350079e-04  8.06513154e-05]
 [-2.03871077e-06 -1.57638470e-05 -1.26555957e-04 ...  6.42808604e-04
   4.79319055e-04  2.93426547e-04]]


In [11]:
# Average Re-referencing via matrix multiplication
MeanMultiplier = np.eye(n_channels) - np.ones((n_channels, n_channels))/n_channels
print(MeanMultiplier)
data_re_ref = np.dot(MeanMultiplier, data)

# # Plot the re-referenced EEG data
# f, axs = plt.subplots(nrows, ncols, figsize=(15, 10))
# axs = axs.flatten()
# for i in range(n_channels):
#     ax = axs.flatten()[i]
#     ax.plot(data_re_ref[i, :])
#     ax.set_title(f"Channel {i+1}")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("Amplitude")

plt.figure()
plt.imshow(MeanMultiplier, aspect='auto')
plt.show()

[[ 0.98484848 -0.01515152 -0.01515152 ... -0.01515152 -0.01515152
  -0.01515152]
 [-0.01515152  0.98484848 -0.01515152 ... -0.01515152 -0.01515152
  -0.01515152]
 [-0.01515152 -0.01515152  0.98484848 ... -0.01515152 -0.01515152
  -0.01515152]
 ...
 [-0.01515152 -0.01515152 -0.01515152 ...  0.98484848 -0.01515152
  -0.01515152]
 [-0.01515152 -0.01515152 -0.01515152 ... -0.01515152  0.98484848
  -0.01515152]
 [-0.01515152 -0.01515152 -0.01515152 ... -0.01515152 -0.01515152
   0.98484848]]


In [12]:
# Creating a fake channel with a polynomial trend
fake_data = raw.get_data()[0]
t = np.linspace(0, 1, len(fake_data))
np.random.seed(42)
trend = np.random.randn(3)
fake_data += 4e-3 * np.polyval(trend, t)

# Let us look at different ways to remove trend from the data
# First, we can simply mean-center the data
# This can be done by removing a 0th order polynomial from the data
trend0 = np.polyfit(t, fake_data, 0)
fake_data_centered = fake_data - np.polyval(trend0, t)

# Second, we can remove a linear trend from the data
# This can be done by removing a 1st order polynomial from the data
trend1 = np.polyfit(t, fake_data, 1)
fake_data_detrended = fake_data - np.polyval(trend1, t)

# Next we can remove a seond order trend from the data
# This can be done by removing a 2nd order polynomial from the data
trend2 = np.polyfit(t, fake_data, 2)
fake_data_detrended2 = fake_data - np.polyval(trend2, t)

# Plot the original data and the detrended data
f, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].plot(t, fake_data)
axs[0, 0].set_title('Original Data')
axs[0, 1].plot(t, fake_data_centered)
axs[0, 1].set_title('Mean-Centered Data')
axs[1, 0].plot(t, fake_data_detrended)
axs[1, 0].set_title('Linear Detrended Data')
axs[1, 1].plot(t, fake_data_detrended2)
axs[1, 1].set_title('Polynomial Detrended Data')
plt.show()

In [52]:
# Compute power at specific frequency for a single channel by taking inner product with a complex sine wave
# Let us compute power at between 50-70 Hz for occipital electrode O1
elecIdx = [elecPos[elecPos['name'] == 'O1'].index[0]][0]

freqs = np.arange(40, 100, 1) 
print(len(freqs))
powerVec = np.zeros(len(freqs))
timeToPlot = 2000 # Time in seconds to plot
t = raw.times[raw.times < timeToPlot]
X = data[elecIdx, raw.times < timeToPlot]
for i, k in enumerate(freqs):
    W = np.exp(-1j * 2 * np.pi * k * t)

    # Compute the inner product (DFT coefficient for frequency k)
    # We use vector multiplication!
    X_k = np.dot(W, X)

    powerVec[i] = np.real(X_k)**2 + np.imag(X_k)**2

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(t, X)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')

plt.subplot(2, 1, 2)
plt.bar(freqs, powerVec)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.show()

60


In [15]:
raw.info

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 66 EEG
 custom_ref_applied: False
 dig: 69 items (3 Cardinal, 66 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 66
 projs: []
 sfreq: 250.0 Hz
>

In [61]:
# Let us compute power at between 5-25 Hz for occipital electrode O1

timeToPlot = 50 # Time in seconds to plot
t = raw.times[raw.times < timeToPlot]
N = len(t)
elecIdx = [elecPos[elecPos['name'] == 'O1'].index[0]][0]
x = np.arange(0,N)/N
X = data[elecIdx, raw.times < timeToPlot]

SR = 250
freqs = np.arange(0, len(X)-1, 1) 
print(len(freqs))
powerVec = np.zeros(len(freqs))
for i, k in enumerate(freqs):
    W = np.exp(-1j * 2 * np.pi * k * x)

    # Compute the inner product (DFT coefficient for frequency k)
    # We use vector multiplication!
    X_k = (W.T @ X)

    powerVec[i] = np.real(X_k)**2 + np.imag(X_k)**2

12499


In [62]:
freq_res = SR/len(X)
freq_vector = freqs*freq_res;
print(max(freq_vector))


plt.plot(powerVec)
plt.xlim([0, 250])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.show()

249.96


In [35]:
freq_res = SR/len(X)
freq_vector = freqs*freq_res;
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(t, X)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')

plt.subplot(2, 1, 2)
plt.bar(freq_vector, powerVec)
plt.xlim([0, 100])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.show()

In [67]:
# We can also compute alpha power using MNE and visualize as a topoplot
freq_range = (8, 12)  # 8-12 Hz

timeToPlot = 2000
t = raw.times[raw.times < timeToPlot]
X = data[:, raw.times < timeToPlot]

# don't use functions. loop through channels instead and do DFT
# psds, freqs = mne.time_frequency.psd_array_multitaper(X, sfreq=raw.info['sfreq'], fmin=freq_range[0], fmax=freq_range[1])
alpha_power = psds.mean(axis=1)

# Exclude the non-EEG channels
picks = mne.pick_types(raw.info, eeg=True, exclude=['leog', 'reog', 'egg', 'audio'])

# Select alpha power for only EEG channels
alpha_power_eeg = alpha_power[picks]
info_eeg = mne.pick_info(raw.info, picks)

# Plot the topomap for alpha power (only using the EEG channels)
mne.viz.plot_topomap(alpha_power_eeg, info_eeg, cmap='RdBu_r', contours=0)

    Using multitaper spectrum estimation with 7 DPSS windows


(<matplotlib.image.AxesImage at 0x7fadf22e8c10>, None)

In [66]:
np.shape(alpha_power_eeg)

(62,)